# Criando a estrutura de diretórios
Cria a estrutura de diretórios do [Behave](https://behave.readthedocs.io/en/latest/)
```
.
│
└───features
    │
    └───steps
```
Onde no diretório <font color='green'>features</font> teremos o arquivo de configuração do ambiente e também os arquivos <em>.feature</em> (que são escritos em [Gherkin](https://www.linkedin.com/pulse/gherkin-comunica%C3%A7%C3%A3o-atrav%C3%A9s-de-um-vocabul%C3%A1rio-e-o-marcus-vin%C3%ADcius/)).
Em <font color='green'>steps</font> escreveremos o código python que executará os passos descritos nos arquivos <em>.feature</em>.

In [1]:
!mkdir -p features/steps

# Configurando o ambiente para o behave
```
.
│
└───features
    └───environment.py
    │
    └───steps
```
No arquivo <font color='green'>environment.py</font> faremos as configurações de ambiente necessárias para execução dos testes. Em <em>before_all</em> faremos os setup dos testes e no <em>after_all</em> faremos o tear-down.

<strong>Obs:</strong> <em>%%writefile</em> é um [comando mágico](https://ipython.readthedocs.io/en/stable/interactive/magics.html#cellmagic-writefile) do [IPython](https://ipython.readthedocs.io/en/stable/overview.html) para escrever um arquivo.

In [2]:
%%writefile ./features/environment.py
from selenium import webdriver

def before_all(context):
    context.browser = webdriver.Chrome()

def after_all(context):
    context.browser.quit()
    

Overwriting ./features/environment.py


# Escrevendo o cenário 1 em Gherkin
```
.
│
└───features
    └───environment.py
    └───google.feature
    │
    └───steps
```
Escreveremos o cenário de teste para a funcionalidade, nesse caso o cenário é checar se a palavra <em>Google</em> está na barra de título do browser.

In [3]:
%%writefile ./features/google.feature
# language:pt

Funcionalidade: Checar título na página do Google

    Cenário: Na página do google deverá ter Google no título
      Dado que o usuário acesse a página "https://www.google.com/"
      Então a palavra "Google" deverá estar no título


Overwriting ./features/google.feature


# Escrevendo o cenário 2 em Gherkin
Nesse cenário, vamos checar se ao fazer uma busca pela palavra <em>Teste</em>, essa palavra deverá aparecer na barra de título do browser.

<strong>Obs:</strong> Quando acrescentamos <em>-a</em> no comando <em>%%writefile</em>, o conteúdo da célula será adicionado ao final do arquivo.

In [4]:
%%writefile -a ./features/google.feature

    Cenário: Ao buscar por Teste no Google, deveremos ter Teste no título
      Dado que o usuário acesse a página "https://www.google.com/"
      Quando inserir a palavra "Teste"
      Então a palavra "Teste" deverá estar no título


Appending to ./features/google.feature


# Criando o cenário 1 no behave
```
.
│
└───features
    └───environment.py
    └───google.feature
    │
    └───steps
        └───google.py
```
Agora implementaremos em python os passos descritos em <em>google.feature</em>, para o primeiro cenário.

In [5]:
%%writefile ./features/steps/google.py
from behave import given, then, when
from selenium.webdriver.common.keys import Keys

@given('que o usuário acesse a página "{link}"')
def acessa_link(context, link):
    context.browser.get(link)


@then('a palavra "{msg}" deverá estar no título')
def checa_titulo(context, msg):
    assert msg in context.browser.title
    

Overwriting ./features/steps/google.py


# Criando o cenário 2 no behave
Implementando os passos descritos em <em>google.feature</em>, para o segundo cenário.

In [6]:
%%writefile -a ./features/steps/google.py
@when('inserir a palavra "{palavra}"')
def checa_titulo(context, palavra):
    element = context.browser.find_element_by_id('lst-ib')
    element.send_keys(palavra + Keys.RETURN)
    assert context.browser.title.startswith(palavra)


Appending to ./features/steps/google.py


# Rodando os testes 
Colocamos <strong>!</strong> para executar um comando shell.
Para rodar os testes do behave, basta digitar comando <strong>behave</strong> seguido do caminho para o diretório <em>features</em>.
No caso abaixo, acrescentamos <strong>--junit</strong> para gerar um relatório. Podemos atribuir o texto que seria exibido no terminal, para uma variável.

In [7]:
!behave --junit ./features

Funcionalidade: Checar título na página do Google # features/google.feature:3

  Cenário: Na página do google deverá ter Google no título       # features/google.feature:5
    Dado que o usuário acesse a página "https://www.google.com/" # features/steps/google.py:4
    Dado que o usuário acesse a página "https://www.google.com/" # features/steps/google.py:4 2.573s
    Então a palavra "Google" deverá estar no título              # features/steps/google.py:9
    Então a palavra "Google" deverá estar no título              # features/steps/google.py:9 0.005s

  Cenário: Ao buscar por Teste no Google, deveremos ter Teste no título  # features/google.feature:9
    Dado que o usuário acesse a página "https://www.google.com/"         # features/steps/google.py:4
    Dado que o usuário acesse a página "https://www.google.com/"         # features/steps/google.py:4 0.419s
    Quando inserir a palavra "Teste"                                     # features/steps/google.py:13
    Quando inserir a p

Convertemos o relatório de <strong>xml</strong> para <strong>html</strong>.

In [8]:
!junit2html ./reports/TESTS-google.xml test_google.html

Exibimos o relatório <strong>html</strong>.

In [9]:
from IPython.core.display import display, HTML
display(HTML('./reports/test_google.html'))

Duration,4.279204 sec
Test Cases,2
Failures,0


## Interação jupyter e shell

A ideia é mostrar que podemos usar dados que seriam exibidos no terminal, de uma maneira mais visual
***

Podemos redirecionar a saída de um comando para uma variável

In [10]:
resultado = !behave --junit ./features

Declaramos uma classe para nos auxiliar com a formatação do que será exibido.

In [11]:
class Color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   RESET = '\033[0m'

Fazemos um loop pela da lista que contém as linhas do resultado. Ao encontrar a string <em>'passed'</em>, colocaremos: 
- passed em verde e negrito, 
- failed em vermelho e negrito, 
- skipped em amarelo e negrito, 

In [12]:
stats = []

for row in resultado:
    if 'passed' in row:
        passed, failed, *skipped = row.split(',')
        skipped = skipped[0]
        stats.append((passed, failed, skipped))
        row = "{}{}{}, {}{}, {}{}{}".format(
            Color.BOLD, Color.GREEN, passed, Color.RED, failed, Color.YELLOW, skipped, Color.RESET)
    print(row)

Funcionalidade: Checar título na página do Google # features/google.feature:3

  Cenário: Na página do google deverá ter Google no título       # features/google.feature:5
    Dado que o usuário acesse a página "https://www.google.com/" # features/steps/google.py:4
    Então a palavra "Google" deverá estar no título              # features/steps/google.py:9

  Cenário: Ao buscar por Teste no Google, deveremos ter Teste no título  # features/google.feature:9
    Dado que o usuário acesse a página "https://www.google.com/"         # features/steps/google.py:4
    Quando inserir a palavra "Teste"                                     # features/steps/google.py:13
    Então a palavra "Teste" deverá estar no título                       # features/steps/google.py:9

1 feature passed,  0 failed,  0 skipped
2 scenarios passed,  0 failed,  0 skipped
5 steps passed,  0 failed,  0 skipped
Took 0m4.208s


Com os dados capturados no loop anterior, calcularemos a porcentagem de sucesso

In [13]:
for passed, failed, skipped in stats:
    
    name = passed.split()[1]
    total = int(passed[0]) + int(failed[1]) + int(skipped[1])
    success = (int(passed[0]) / total) * 100
    print('{} : {}% passed'.format(name.title(), success))

Feature : 100.0% passed
Scenarios : 100.0% passed
Steps : 100.0% passed


De acordo com a porcentagem, definimos o deploy

In [14]:
if success > 90:
    deploy_status = "<font size=8 color='green'>Deploy Aprovado!</font>"
elif success > 70:
    deploy_status = "<font size=8 color='yellow'>Deploy a ser avaliado</font>"
else:
    deploy_status = "<font size=8 color='red'>Deploy Reprovado!</font>"

display(HTML('{}'.format(deploy_status)))